In [1]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.persistence.models import Trial
from noworkflow.now.models.cleaning.history import *
import numpy as np
from pprint import pprint

In [2]:
import numpy as np

def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Evaluation((trial_id, evaluation_id))

def get_act_evals(activation_id):
    activation = get_activation(activation_id)
    return activation.evaluations    

def print_act_forward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependent_dependencies):
        pprint.pprint(i.to_dict())
        
def print_act_backward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependency_dependencies):
        pprint.pprint(i.to_dict())

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())
        


In [3]:
def get_precedent(evaluations, options=None):
    """Return code components that contribute to the creation of a given evaluation"""
    result = copy(evaluations)
    querier = DependencyQuerier(options)
    visited = set()
    code_component_op = {}
    operations = set()
    while result:
        nodes_to_visit, visited, found = querier.navigate_dependencies(result, visited=visited)
        result = []
        component_to_visit = []
        new_ops = set()
        # Extract operations from visited evaluations
        for context in visited:
            evaluation = context.evaluation

            # Evaluation operations
            component = evaluation.code_component
            operation = find_types(component, code_component_op)
            if operation and operation not in operations:
                new_ops.add(operation)
                operations.add(operation)
        
        # Expand dependencies to all evaluations of new operations
        for operation in new_ops:
            if operation.this_block:
                result.extend(operation.this_block.recursive_evaluations())

    #return sorted(operations, key=lambda c: c.id)
    return operations

In [4]:
def find_types(code_component, found=None):
    """Find specific component typesfrom code_components"""
    found = found or {}
    if code_component in found:
        pass
    elif code_component.type in ('call', 'name'):
        found[code_component] = code_component
    elif not code_component.container_id:
        found[code_component] = None
    else:
        found[code_component] = find_cell(code_component.container.this_component, found=found)
    return found[code_component]

In [5]:
def print_precedents(evals):
    code_componenents = get_precedent([evals])
    
    for code_comp in list(code_componenents):
        obj_eval = Evaluation((code_comp.trial_id, code_comp.id))
        obj_act = Activation((obj_eval.trial_id, obj_eval.activation_id))

        if code_comp.type in ('name'):
            if code_comp.mode == 'w':
                print(code_comp.name, obj_eval.repr)
            if (code_comp.type == 'call') and (obj_eval.repr != 'None'):
                print(code_comp.name)

In [6]:
def module(num): 
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean((x - y))

def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))

In [7]:
var_a = 10

In [8]:
var_b = -100

In [9]:
rme_err = mae(var_a, var_b)

In [10]:
var_c = 4869

In [11]:
var_d = var_c * 234
var_c = var_d / 3

In [12]:
var_e = mae(var_c, var_b)

In [13]:
#k = module(rmse_err)
var_final = now_variable('var_final', rmse(var_c, var_e))

Activation(id=103, name=now_variable, start_checkpoint=31.310220231, code_block_id=-1)
[now_variable(func), 'var_final'(argument), rmse(var_c, var_e)(argument)] - []
Evaluation(id=88, checkpoint=31.310198318999998, code_component_id=1020, activation_id=81, repr=100.0)


In [14]:
evals = get_evaluation(103)
a = get_precedent([evals])


In [18]:
get_precedent([evals])

{code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1002, 'var_e', 'name', 'w', 1, 0, 1, 5, 999).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1003, 'mae(var_c, var_b)', 'call', 'r', 1, 8, 1, 25, 999).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1007, 'mae', 'name', 'r', 1, 8, 1, 11, 999).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1008, 'var_c', 'name', 'r', 1, 12, 1, 17, 999).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1009, 'var_b', 'name', 'r', 1, 19, 1, 24, 999).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1010, '<ipython-input-13-d5e600da0c6a>', 'cell', 'w', 1, 0, 3, 0, nil).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1013, 'var_final', 'name', 'w', 2, 0, 2, 9, 1010).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1014, 'now_variable(''var_final'', rmse(var_c, var_e))', 'call', 'r', 2, 12, 2, 57, 1010).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1018, 'now_variable', 'name', 'r', 2, 12, 2, 2

In [17]:
print_precedents(evals)

var_b code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 945, 'np', 'name', 'r', 11, 19, 11, 21, 922).
var_e evaluation(6bf7815e-aa57-4896-bc65-01f0d336a605, 63, 31.062937607, 997, 57, '3', 63).
var_final {}
var_d code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 945, 'np', 'name', 'r', 11, 19, 11, 21, 922).
var_a code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 945, 'np', 'name', 'r', 11, 19, 11, 21, 922).
var_c code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 945, 'np', 'name', 'r', 11, 19, 11, 21, 922).
var_c code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 945, 'np', 'name', 'r', 11, 19, 11, 21, 922).


In [105]:
op_dict = {}
for i in reversed(list(a)):
    code_comp = i
    obj_eval = Evaluation((i.trial_id, i.id))
    obj_act = Activation((obj_eval.trial_id, obj_eval.activation_id))
    
    if  ((code_comp.type == 'call') and (code_comp.mode == 'r') and (obj_eval.repr == "{}")) or \
        (code_comp.type == 'name' and code_comp.mode == 'w'): 
            op_dict[round(float(obj_eval.checkpoint), 6)] = [round(float(obj_act.start_checkpoint),4), round(float(obj_eval.checkpoint), 4), code_comp.name, code_comp.type, code_comp.mode]    

In [106]:
op_dict

{38.655931: [38.143, 38.6559, 'var_c', 'name', 'w'],
 38.657859: [38.6578,
  38.6579,
  "now_variable('var_final', rmse(var_c, var_e))",
  'call',
  'r'],
 38.655414: [38.655, 38.6554, 'var_c', 'name', 'w'],
 38.654817: [38.143, 38.6548, 'var_a', 'name', 'w'],
 38.655648: [38.143, 38.6556, 'var_d', 'name', 'w'],
 38.657847: [38.6578, 38.6578, 'var_final', 'name', 'w'],
 38.652183: [38.143, 38.6522, 'np.mean((x - y))', 'call', 'r'],
 38.656258: [38.143, 38.6563, 'var_e', 'name', 'w'],
 38.655585: [38.143, 38.6556, 'var_b', 'name', 'w']}

In [128]:
op_dict = {}
for i in reversed(list(a)):
    code_comp = i
    obj_eval = Evaluation((i.trial_id, i.id))
    obj_act = Activation((obj_eval.trial_id, obj_eval.activation_id))
    
    if ((code_comp.type == 'call') and (code_comp.mode == 'r')) or \
        (code_comp.type == 'name' and code_comp.mode == 'r'):
        op_dict[round(float(obj_eval.checkpoint), 6)] = [round(float(obj_act.start_checkpoint),4), round(float(obj_eval.checkpoint), 4), code_comp.name, code_comp.type, code_comp.mode]    

In [129]:
pprint(op_dict)

{38.652183: [38.143, 38.6522, 'np.mean((x - y))', 'call', 'r'],
 38.652281: [38.6522, 38.6523, 'np', 'name', 'r'],
 38.652315: [38.6522, 38.6523, 'x', 'name', 'r'],
 38.652336: [38.6522, 38.6523, 'y', 'name', 'r'],
 38.652749: [38.6522, 38.6527, 'np.sqrt(np.mean((x - y)**2))', 'call', 'r'],
 38.652961: [38.143, 38.653, 'np', 'name', 'r'],
 38.652985: [38.143, 38.653, 'np.mean((x - y)**2)', 'call', 'r'],
 38.653203: [38.143, 38.6532, 'np', 'name', 'r'],
 38.653406: [38.143, 38.6534, 'x', 'name', 'r'],
 38.653437: [38.143, 38.6534, 'y', 'name', 'r'],
 38.655238: [38.655, 38.6552, 'var_a', 'name', 'r'],
 38.655252: [38.655, 38.6553, 'var_b', 'name', 'r'],
 38.655771: [38.143, 38.6558, 'var_c', 'name', 'r'],
 38.65608: [38.143, 38.6561, 'var_d', 'name', 'r'],
 38.656294: [38.143, 38.6563, 'mae(var_c, var_b)', 'call', 'r'],
 38.657721: [38.143, 38.6577, 'mae', 'name', 'r'],
 38.657752: [38.143, 38.6578, 'var_c', 'name', 'r'],
 38.657859: [38.6578,
             38.6579,
             "now_var